In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os

# Load the model from .h5 file
model = keras.models.load_model('D:\AMRITA COLLEGE\SEM 6\Maths\Project\discriminator500.h5')

# Freeze the layers of the pre-trained model
for layer in model.layers:
    layer.trainable = False

# Add new layers for classification
input_layer = keras.Input(shape=(28, 28, 1))
x = keras.layers.Conv2D(32, (3, 3), activation='relu')(input_layer)
x = keras.layers.MaxPooling2D((2, 2))(x)
x = keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = keras.layers.MaxPooling2D((2, 2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
output_layer = keras.layers.Dense(2, activation='softmax')(x)

# Combine the pre-trained model and new layers
model = keras.Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Prepare data
MAIN_DIR = "xyz"

def load_images(folder):
    imgs = []
    labels = []
    for i, class_name in enumerate(os.listdir(folder)):
        subdir = os.path.join(folder, class_name)
        for img_name in os.listdir(subdir):
            img_path = os.path.join(subdir, img_name)
            try:
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                imgs.append(img)
                labels.append(i)
            except:
                continue
    
    imgs = np.array(imgs)
    labels = np.array(labels)

    return imgs, labels

data, labels = load_images(MAIN_DIR)

labels = tf.keras.utils.to_categorical(labels, num_classes=2) # Convert labels to one-hot encoded format
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
num_epochs = 10
batch_size = 32
history = model.fit(train_data, train_labels, epochs=num_epochs, batch_size=batch_size, validation_data=(test_data, test_labels))

# Plot accuracy curve
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss curve
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
